In [1]:
import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
gparentdir = os.path.dirname(os.path.dirname(currentdir))
sys.path.insert(0, gparentdir)
import scobra
import get_all_functions as gaf
import re

new=scobra.Model('sample/Diel_Leaf_Model_cleaned.xls')
new= scobra.model(new)
#Maximize Sucrose output (objective)
#vary light (scan)
#allow CO2 and O2, even Proton and Water
#Output maintenance ATP and NADPHox
tx_and_biomass = []
for i in new.Reactions(): 
    if '_tx' in i:
        tx_and_biomass.append(re.search('(.*)_tx', i).group())
    elif '_biomass' in i: 
        tx_and_biomass.append(re.search('(.*)_biomass', i).group())
#tx_and_biomass.append(re.search('(.*)_tx', 'slkdfjd_tx').group())
tx_and_biomass

necessary_rxn= ['ATPase_tx', 'CO2_tx', 'Photon_tx', 'O2_tx','H2O_tx', 'H_tx','NADPHoxc_tx','NADPHoxm_tx','NADPHoxp_tx','Sucrose_tx']
unecessary_rxn= list(set(tx_and_biomass) - set(necessary_rxn))

#blocking _tx and _biomass reactions that are not considered
for i in unecessary_rxn: 
    new.GetReaction(i).lower_bound = 0.0
    new.GetReaction(i).upper_bound = 0.0

#freeing _tx and _biomass reactions that are considered
for i in necessary_rxn: 
    if new.GetReaction(i).lower_bound == 0.0:
        new.GetReaction(i).upper_bound == 1000.0
    else: 
        new.GetReaction(i).lower_bound == 1000.0
        new.GetReaction(i).upper_bound == 1000.0

#setting constraints
#new.SetConstraint('ATPase_tx', 90, 90)
#new.SetConstraint('NADPHoxc_tx', 10, 10)
#new.SetConstraint('NADPHoxm_tx', 10, 10)
#new.SetConstraint('NADPHoxp_tx', 10, 10)
#setting constraints for sucrose entering and exiting cytoplasm
new.SetConstraint('Sucrose_ec', -1, -1)

#setting objectives
new.SetObjective('Photon_tx')
new.SetObjDirec('Min')
#new.SetFixedFlux({'unlProtHYPO_c': 0})

#running scans
#new.MinFluxSolve() 
#new.Solve()

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '(  )' for <Reaction HEXOKINASE_RXN_MANNOSE_c at 0x7f5f630fa5d0>


In [2]:
new.MinFluxSolve()

optimal
[<Reaction 10_FORMYL_THF_pc_reverse at 0x7f5f9c5bb150>, <Reaction 1TRANSKETO_RXN_p_reverse at 0x7f5f9c5bb250>, <Reaction 1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p_reverse at 0x7f5f9c5bb290>, <Reaction 2KG_ACONITATE_mc_reverse at 0x7f5f9c5bb910>, <Reaction 2KG_CIT_mc_reverse at 0x7f5f9c5bb950>, <Reaction 2KG_ISOCITRATE_mc_reverse at 0x7f5f9c5bb9d0>, <Reaction 2KG_MAL_mc_reverse at 0x7f5f9c5bba10>, <Reaction 2KG_MAL_pc_reverse at 0x7f5f9c5bba50>, <Reaction 2KG_OAA_mc_reverse at 0x7f5f9c5bba90>, <Reaction 2KG_SUC_mc_reverse at 0x7f5f9c5bbad0>, <Reaction 2PGADEHYDRAT_RXN_c_reverse at 0x7f5f9c5bbb50>, <Reaction 2PGADEHYDRAT_RXN_p_reverse at 0x7f5f9c5bbbd0>, <Reaction 2TRANSKETO_RXN_p_reverse at 0x7f5f9c5bbc50>, <Reaction 2_KETOGLUTARATE_xc_reverse at 0x7f5f9c5bbcd0>, <Reaction 2_PERIOD_7_PERIOD_1_PERIOD_90_RXN_c_reverse at 0x7f5f9c5bbd10>, <Reaction 2_PERIOD_7_PERIOD_7_PERIOD_44_RXN_c_reverse at 0x7f5f62dff310>, <Reaction 3PGAREARR_RXN_c_reverse at 0x7f5f62dff210>, <Reaction 3PGAREARR_RXN

In [3]:
new.GetSol()

{'1TRANSKETO_RXN_p': 4.000000000000003,
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi': 1.4333333333331797,
 '1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p': 25.4333333333332,
 '1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p': 24.00000000000001,
 '2TRANSKETO_RXN_p_reverse': 4.000000000000003,
 'ATP_ADP_Pi_pc_reverse': 3.2999999999995366,
 'ATP_ADP_mc': 4.299999999999528,
 'CO2_ec': 12.000000000000007,
 'CO2_pc_reverse': 12.000000000000009,
 'CO2_tx': 12.000000000000007,
 'CYTOCHROME_C_OXIDASE_RXN_mi': 0.7166666666665898,
 'DHAP_Pi_pc': 2.0,
 'F16ALDOLASE_RXN_c_reverse': 2.0,
 'F16ALDOLASE_RXN_p_reverse': 3.9999999999999987,
 'F16BDEPHOS_RXN_c': 2.0,
 'F16BDEPHOS_RXN_p': 3.9999999999999987,
 'GAP_Pi_pc': 1.9999999999999996,
 'GLUC1PURIDYLTRANS_RXN_c': 1.0,
 'H2O_ec': 11.000000000000018,
 'H2O_mc': 5.73333333333272,
 'H2O_pc_reverse': 12.733333333332737,
 'H2O_tx': 11.000000000000018,
 'H_pc': 3.299999999999555,
 'H_tx': 1.0,
 'INORGPYROPHOSPHAT_RXN_c': 1.0000000000000009,
 'MALATE_DEHYDROGENASE_NADPs_RXN_p': 1.433

In [5]:
new.solution.x_dict

pandas.core.series.Series

In [14]:
this = new.GetSol()
this

{'1TRANSKETO_RXN_p': 3.9999999999999987,
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi': 1.4333333333328042,
 '1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p': 25.433333333332804,
 '1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p': 23.999999999999993,
 '2TRANSKETO_RXN_p_reverse': 3.9999999999999987,
 'ATP_ADP_Pi_pc_reverse': 3.2999999999984184,
 'ATP_ADP_mc': 4.299999999998416,
 'CO2_ec': 11.999999999999998,
 'CO2_pc_reverse': 11.999999999999996,
 'CO2_tx': 11.999999999999998,
 'CYTOCHROME_C_OXIDASE_RXN_mi': 0.7166666666664021,
 'DHAP_Pi_pc': 1.0,
 'F16ALDOLASE_RXN_c_reverse': 0.9999999999999994,
 'F16ALDOLASE_RXN_p_reverse': 4.999999999999999,
 'F16BDEPHOS_RXN_c': 0.9999999999999998,
 'F16BDEPHOS_RXN_p': 4.999999999999999,
 'G6P_Pi_pc': 1.0000000000000004,
 'GAP_Pi_pc': 1.0000000000000007,
 'GLUC1PURIDYLTRANS_RXN_c': 1.0,
 'H2O_ec': 11.000000000000009,
 'H2O_mc': 5.733333333331215,
 'H2O_pc_reverse': 13.733333333331226,
 'H2O_tx': 11.000000000000009,
 'H_pc': 3.2999999999983993,
 'H_tx': 1.0,
 'INORGPYROPHOSPHAT_R

In [ ]:
that = new.GetSol()

In [25]:
new.GetConstraints()

{'10_FORMYL_THF_pc': (-1000.0, 1000.0),
 '1TRANSKETO_RXN_p': (-1000.0, 1000.0),
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mc': (0.0, 1000.0),
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi': (0.0, 1000.0),
 '1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p': (-1000.0, 1000.0),
 '1_PERIOD_1_PERDIOD_1_PERIOD_255_RXN_c': (0.0, 1000.0),
 '1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p': (0.0, 1000.0),
 '1_PERIOD_2_PERIOD_1_PERIOD_2_RXN_m': (0.0, 1000.0),
 '1_PERIOD_2_PERIOD_1_PERIOD_2_RXN_p': (0.0, 1000.0),
 '1_PERIOD_2_PERIOD_1_PERIOD_9_RXN_c': (0.0, 1000.0),
 '1_PERIOD_5_PERIOD_1_PERIOD_20_RXN_c': (0.0, 1000.0),
 '1_PERIOD_8_PERIOD_4_PERIOD_9_RXN_p': (0.0, 1000.0),
 '1_PERIOD_8_PERIOD_5_PERIOD_1_RXN_p': (0.0, 1000.0),
 '2KG_ACONITATE_mc': (-1000.0, 1000.0),
 '2KG_CIT_mc': (-1000.0, 1000.0),
 '2KG_ISOCITRATE_mc': (-1000.0, 1000.0),
 '2KG_MAL_mc': (-1000.0, 1000.0),
 '2KG_MAL_pc': (-1000.0, 1000.0),
 '2KG_OAA_mc': (-1000.0, 1000.0),
 '2KG_SUC_mc': (-1000.0, 1000.0),
 '2OXOGLUTARATEDEH_RXN_m': (0.0, 1000.0),
 '2PGADEHYDRAT_RXN_

In [17]:
new.Solve()

optimal


In [23]:
new.GetState()['solution'].x_dict
new.GetSol()

{'1TRANSKETO_RXN_p': 3.9999999999976397,
 '1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi': 1.433333333332601,
 '1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p': 25.433333333318352,
 '1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p': 20.699999999987796,
 '2PGADEHYDRAT_RXN_c': -4.733333333330566,
 '2PGADEHYDRAT_RXN_p': 4.733333333330524,
 '2TRANSKETO_RXN_p': -3.99999999999764,
 '2_PERIOD_7_PERIOD_1_PERIOD_90_RXN_c': -9.999999999994092,
 '3PGAREARR_RXN_c': -4.733333333330566,
 '3PGAREARR_RXN_p': 4.733333333330524,
 'ADENYL_KIN_RXN_c': -2.149999999998842,
 'ADENYL_KIN_RXN_m': 2.1499999999988604,
 'ATP_AMP_mc': 2.149999999998844,
 'CO2_ec': 11.999999999992848,
 'CO2_pc': -11.99999999999289,
 'CO2_tx': 11.999999999992848,
 'CYTOCHROME_C_OXIDASE_RXN_mi': 0.7166666666663005,
 'F16ALDOLASE_RXN_c': -9.999999999994094,
 'FUMHYDR_RXN_c': -1.4333333333324938,
 'FUMHYDR_RXN_m': 1.4333333333324938,
 'G6P_Pi_pc': -7.999999999995276,
 'GAPOXNPHOSPHN_RXN_c': -3.2999999999980028,
 'GAP_3PGA_pc': 1.4333333333325455,
 'GAP_Pi_pc': 15.266

In [5]:
new.MinFluxSolve()

optimal


In [15]:
new.solution.x_dict

10_FORMYL_THF_pc                        -1.900431e-15
1TRANSKETO_RXN_p                         4.000000e+00
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mc     2.213250e-15
1_PERIOD_10_PERIOD_2_PERIOD_2_RXN_mi     1.433333e+00
1_PERIOD_18_PERIOD_1_PERIOD_2_RXN_p      2.543333e+01
1_PERIOD_1_PERDIOD_1_PERIOD_255_RXN_c    0.000000e+00
1_PERIOD_2_PERIOD_1_PERIOD_13_RXN_p      2.400000e+01
1_PERIOD_2_PERIOD_1_PERIOD_2_RXN_m       0.000000e+00
1_PERIOD_2_PERIOD_1_PERIOD_2_RXN_p       0.000000e+00
1_PERIOD_2_PERIOD_1_PERIOD_9_RXN_c       0.000000e+00
1_PERIOD_5_PERIOD_1_PERIOD_20_RXN_c      0.000000e+00
1_PERIOD_8_PERIOD_4_PERIOD_9_RXN_p       0.000000e+00
1_PERIOD_8_PERIOD_5_PERIOD_1_RXN_p       0.000000e+00
2KG_ACONITATE_mc                         0.000000e+00
2KG_CIT_mc                               0.000000e+00
2KG_ISOCITRATE_mc                        0.000000e+00
2KG_MAL_mc                               0.000000e+00
2KG_MAL_pc                              -1.344675e-14
2KG_OAA_mc                  